# AymaraAI Accuracy Example

This notebook demonstrates:
- Creating an eval with AymaraSDK
- Fetching eval prompts
- Calling OpenAI with those prompts
- Creating an eval run with the responses

## Requirements
- Set `OPENAI_API_KEY` and `AYMARA_AI_API_KEY` in your environment or `.env` file.
- Install dependencies: `pip install openai aymara-ai dotenv`

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Environment and imports
import os

import openai
import pandas as pd
from dotenv import load_dotenv

from aymara_ai import AymaraAI

pd.set_option("display.max_colwidth", None)


load_dotenv()

## Set up API keys

In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError("OPENAI_API_KEY not set in environment.")
openai.api_key = OPENAI_API_KEY

## Instantiate the AymaraSDK client

In [ ]:
client = AymaraAI()

## Create an eval

In [ ]:
ai_description = "A helpful AI assistant."

knowledge_base = open("aymara_language.txt").read()

eval_obj = client.evals.create(
    ai_description=ai_description,
    eval_type="accuracy",
    name="basic accuracy eval",
    ground_truth=knowledge_base,
    num_prompts=5,
)
eval_id = eval_obj.eval_uuid
eval_obj

## Fetch prompts for the eval

In [ ]:
from aymara_ai.lib.async_utils import wait_until_complete

eval_obj = wait_until_complete(client.evals.get, resource_id=eval_id)
eval_obj

In [ ]:
from typing import List

from aymara_ai.types.eval_prompt import EvalPrompt

prompts_response = client.evals.list_prompts(eval_id)
prompts: List[EvalPrompt] = prompts_response.items
prompts

## Call OpenAI for each prompt and collect responses

In [ ]:
from typing import List

from aymara_ai.types.eval_response_param import EvalResponseParam


def answer_prompts(prompts: List[EvalPrompt]) -> List[EvalResponseParam]:
    """Answer the prompts using OpenAI's API."""
    responses: List[EvalResponseParam] = []
    system_prompt = f"""<role>Assume this role for the following task: [{ai_description}].</role><task>Answer user questions using only the information in the knowledge base. If the knowledge base lacks the full answer to the question, then reply that you do not know the answer to the question. Do not share information outside the knowledge base.</task><knowledge_base>{knowledge_base}</knowledge_base>"""

    for prompt in prompts:
        prompt_text = prompt.content
        prompt_uuid = prompt.prompt_uuid
        completion = openai.responses.create(
            model="o4-mini",
            instructions=system_prompt,
            input=prompt_text,
            max_output_tokens=256,
        )
        answer = completion.output_text
        responses.append(EvalResponseParam(content=answer, prompt_uuid=prompt_uuid, ai_refused=not answer))
    return responses

In [ ]:
responses = answer_prompts(prompts)
responses

## Create an eval run with the responses

In [ ]:
eval_run = client.evals.runs.create(eval_uuid=eval_id, responses=responses)
eval_run_id = eval_run.eval_run_uuid
eval_run

In [ ]:
eval_run = wait_until_complete(client.evals.runs.get, resource_id=eval_run_id)
eval_run

## Create a Report for the Runs

In [ ]:
all_runs = [eval_run]
report = client.reports.create(eval_run_uuids=[run.eval_run_uuid for run in all_runs])
report

In [ ]:
report = wait_until_complete(client.reports.get, resource_id=report.eval_suite_report_uuid)
report

In [ ]:
rows = [s.to_dict() for s in report.eval_run_reports]

df = pd.DataFrame(rows)
df

In [ ]:
from aymara_ai.lib.plot import graph_eval_stats  # type: ignore

graph_eval_stats(eval_runs=all_runs)